In [1]:
import os
os.chdir("../")

In [6]:
#entity
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    

In [7]:
#Config
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directory

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifact_root])
        
    def data_transformation_config(self) ->DataTransformationConfig:
        config = self.config.data_transformation
        create_directory([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
        )
        return data_transformation_config
    


    

In [12]:
import os
from TextSummarizer.logging import  logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk



In [15]:
#componets
class DataTransformation:
    def __init__(self,
                 config:DataTransformationConfig):
        
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert_examples_to_features(self,example_batch):
        
        input_encodings = self.tokenizer(example_batch["dialogue"],max_length=1024,truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"],max_length=128,truncation=True)
        
        return {
            "input_ids":input_encodings["input_ids"],
            "attention_mask":input_encodings["attention_mask"],
            "labels":target_encodings["input_ids"]
        }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
    

In [16]:
#pipeline

try:
    config = ConfigurationManager()
    data_transformation_config = config.data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-11-22 19:45:20,921: INFO: common: Directory Artifacts created]
[2023-11-22 19:45:20,929: INFO: common: Directory Artifacts/data_transformation created]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]